![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare_jsl/ER_CLINICAL_ABBREVIATION_ACRONYM.ipynb)

# `sbiobertresolve_clinical_abbreviation_acronym` **Models**

This model maps clinical abbreviations and acronyms to their meanings using `sbiobert_base_cased_mli` Sentence Bert Embeddings. This model is an improved version of the base model, and includes more variational data.

## 1. Colab Setup

**Import license keys**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## 2. Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

## 3. Select the model and construct the pipeline

In [ ]:
MODEL_NAME = "sbiobertresolve_clinical_abbreviation_acronym"

**Create the pipeline**

In [ ]:
document_assembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
      .setInputCols(["document"])\
      .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("word_embeddings")

clinical_ner = medical.NerModel.pretrained("ner_abbreviation_clinical", "en", "clinical/models") \
      .setInputCols(["document", "token", "word_embeddings"]) \
      .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
      .setInputCols(["document", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['ABBR'])

sentence_chunk_embeddings = medical.BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
      .setInputCols(["document", "ner_chunk"])\
      .setOutputCol("sentence_embeddings")\
      .setChunkWeight(0.5)\
      .setCaseSensitive(True)

abbr_resolver = medical.SentenceEntityResolverModel.pretrained(MODEL_NAME, "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("abbr_meaning")\
      .setDistanceFunction("EUCLIDEAN")\
    

resolver_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        sentence_chunk_embeddings,
        abbr_resolver
  ])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_abbreviation_clinical download started this may take some time.
[OK!]
sbiobert_base_cased_mli download started this may take some time.
[OK!]
sbiobertresolve_clinical_abbreviation_acronym download started this may take some time.
[OK!]


## 4. Create example inputs

In [ ]:
sample_text = [
"""The patient admitted from the IR for aggressive irrigation of the Miami pouch. DISCHARGE DIAGNOSES: 1. A 58-year-old female with a history of stage 2 squamous cell carcinoma of the cervix status post total pelvic exenteration in 1991.""",

"""Gravid with estimated fetal weight of 6-6/12 pounds. LOWER EXTREMITIES: No edema. LABORATORY DATA: Laboratory tests include a CBC which is normal. 
    Blood Type: AB positive. Rubella: Immune. VDRL: Nonreactive. Hepatitis C surface antigen: Negative. HIV: Negative. One-Hour Glucose: 117. Group B strep has not been done as yet."""
 ]

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text, StringType()).toDF('text')
df.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|The patient admitted from the IR for aggressive irrigation of the Miami pouch. DISCHARGE DIAGNOSE...|
|Gravid with estimated fetal weight of 6-6/12 pounds. LOWER EXTREMITIES: No edema. LABORATORY DATA...|
+----------------------------------------------------------------------------------------------------+



## 5. Use the pipeline to create outputs

In [ ]:
result = resolver_pipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.begin, 
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata, 
                                     result.abbr_meaning.result, 
                                     result.abbr_meaning.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("entity"),
              F.expr("cols['4']").alias("abbr_meaning"),
              F.expr("cols['5']['all_k_resolutions']").alias("all_k_resolutions")).show(truncate = 50)

+-----+-----+---+------+------------------------------------+--------------------------------------------------+
|chunk|begin|end|entity|                        abbr_meaning|                                 all_k_resolutions|
+-----+-----+---+------+------------------------------------+--------------------------------------------------+
|   IR|   30| 31|  ABBR|            interventional radiology|interventional radiology:::immediate-release:::...|
|  CBC|  126|128|  ABBR|                Complete Blood Count|Complete Blood Count:::Complete blood count:::b...|
|   AB|  164|165|  ABBR|           blood group in ABO system|              blood group in ABO system:::abortion|
| VDRL|  194|197|  ABBR|Venereal disease research laboratory|Venereal disease research laboratory:::venous b...|
|  HIV|  252|254|  ABBR|        human immunodeficiency virus|human immunodeficiency virus:::blood group in A...|
+-----+-----+---+------+------------------------------------+-----------------------------------

## 6. Visualize results

In [ ]:
from sparknlp_display import AssertionVisualizer

assertion_vis = AssertionVisualizer()

for i in range(len(sample_text)):
    assertion_vis.display(result = result.collect()[i], label_col = "ner_chunk", assertion_col = "abbr_meaning")